In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from datetime import date
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler

In [4]:
file_path2 = Path("../../../data/cleandata/1hot_merged_post_ICO.csv")
df = pd.read_csv(file_path2)
df.head()

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,Ethereum,ETH,0,0,0.460570,0.061160,0.027390,0.901510,100,779,...,0,0,0,0,0,0,0,0,0,0
1,Binance Coin,BNB,20,0,0.563773,0.096670,0.041410,0.861890,100,848,...,0,0,0,0,0,0,0,0,0,0
2,EOS,EOS,350,0,0.828470,0.032200,0.858550,0.109270,100,522,...,0,0,0,0,0,0,1,0,0,0
3,Cardano,ADA,488,0,0.589474,0.098387,0.032946,0.868753,93,1049,...,0,0,0,0,0,0,1,0,0,0
4,Huobi Token,HT,0,0,0.525717,0.081630,0.033480,0.884940,100,779,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Create our features
X = df.drop(columns=["success", "Longevity", "Name", "Ticker"])

# Create our target
y = df["success"].values.reshape(-1, 1)

In [6]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
# Scale the Features Data
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
p = pd.DataFrame(X_train_scaled)

In [31]:
q = p.isna()
q == True

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
398,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
399,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
400,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
import math
for p in X_train_scaled:
    if math.isnan(p)
    print(p)

SyntaxError: invalid syntax (<ipython-input-14-ad1760759b5c>, line 3)

In [12]:
# Balanced Random Forest Classifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

brf_model = brf.fit(X_train_scaled, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [85]:
y_pred_brf = brf.predict(X_test_scaled)

In [86]:
# Calculated the balanced accuracy score
bac_brf = balanced_accuracy_score(y_test, y_pred_brf)

print(f"The Balanced Accuracy Score for the Balanced Random Forest Classifier is: {bac_brf}")

The Balanced Accuracy Score for the Balanced Random Forest Classifier is: 0.974820143884892


In [87]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_brf)

array([[132,   7],
       [  0,   5]], dtype=int64)

In [88]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.95      1.00      0.97      0.97      0.94       139
          1       0.42      1.00      0.95      0.59      0.97      0.95         5

avg / total       0.98      0.95      1.00      0.96      0.97      0.95       144



In [89]:
# List the features sorted in descending order by feature importance
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.1579653340033315, 'price'),
 (0.13249107734637341, 'positive'),
 (0.10988532724996956, 'compound'),
 (0.10286294029074394, 'neutral'),
 (0.0951373090493702, 'End_2017-09-28 03:00:00'),
 (0.0860139556161752, 'platform_Ethereum'),
 (0.06870000934711201, 'country_United States of America'),
 (0.06794094642208093, 'Start_2017-09-28 03:00:00'),
 (0.06714144086915647, 'negative'),
 (0.031746031746031744, 'country_Switzerland'),
 (0.02715210966484312, 'country_Singapore'),
 (0.022222222222222223, 'country_Canada'),
 (0.019567032388128538, 'no_of_posts'),
 (0.006302083333333331, 'country_Russia'),
 (0.004872180451127816, 'country_Gibraltar'),
 (0.0, 'platform_Waves'),
 (0.0, 'platform_Stellar'),
 (0.0, 'platform_Scrypt'),
 (0.0, 'platform_Own Blockchain'),
 (0.0, 'platform_Omni'),
 (0.0, 'platform_Nxt'),
 (0.0, 'platform_NEO'),
 (0.0, 'platform_NEM'),
 (0.0, 'platform_Litecoin'),
 (0.0, 'platform_Hyperledger'),
 (0.0, 'platform_Graphene'),
 (0.0, 'platform_EOS'),
 (0.0, 'platform_DAG'),
 (